In [1]:
import sys
sys.path.append('../../30_data_tools/')
sys.path.append('../process_masks/')

In [2]:
from pathlib import Path
from PIL import Image
from helper import load_dotenv
from file_interaction import get_related_filepath, get_generic_image_filepath, open_img
from mask_functions import load_masks, get_whole_mask
import numpy as np
from scipy.ndimage import gaussian_filter
from random import randint
from tqdm.auto import tqdm
import pandas as pd
import sqlite3
import cv2

In [3]:
import plotly.express as px

In [4]:
Image.MAX_IMAGE_PIXELS = None

In [5]:
IMG_SIZE = (224,224)
MIN_MASK_PARTIAL = 0.25

In [6]:
def get_fft( input_img ):
    ft = np.fft.ifftshift( input_img )
    ft = np.fft.fft2(ft)
    ft = np.fft.fftshift(ft)
    
    return ft

In [7]:
def cut_img_to_tiles( img, relevant_box, mask=None, is_polygon_box=False ):
    # convolve over image
    out = []
    conv_size = IMG_SIZE
    step_horizontal = int(round(conv_size[0] / 2))
    step_vertical = int(round(conv_size[1] / 2))
    
    for left in range(0, img.shape[1] - step_horizontal, step_horizontal):
        for top in range(0, img.shape[0] - step_vertical, step_vertical):
            
            if left + conv_size[0] > img.shape[1]:
                left = img.shape[1] - conv_size[0]

            if top + conv_size[1] > img.shape[0]:
                top = img.shape[0] - conv_size[1]

            if (mask is None) == False:
                mask_partial = mask[
                    top:top+conv_size[1],
                    left:left+conv_size[0]
                ].mean()   
            elif is_polygon_box:
                p_label = Polygon(relevant_box)
                p_tile = Polygon([
                    (left,top),
                    (left + step_horizontal,top),
                    (left + step_horizontal,top + step_vertical),
                    (left,top + step_vertical)
                ])

                mask_partial = p_label.intersection(p_tile).area / p_tile.area
            else:
                # ist keine Maske vorhanden wird die Intersection
                # mit der Moirebox verwendet
                intersection_box = [
                    max([relevant_box[0], left]),
                    max([relevant_box[1], top]),
                    min([relevant_box[0] + relevant_box[2], left+conv_size[0]]),
                    min([relevant_box[1] + relevant_box[3], top+conv_size[1]])
                ]
        
                if intersection_box[0] <= intersection_box[2] and intersection_box[1] <= intersection_box[3]:
                    mask_partial = ((intersection_box[2] - intersection_box[0]) * (intersection_box[3] - intersection_box[1]))  / (conv_size[0] * conv_size[1])      
                else:
                    mask_partial = 0

            if mask_partial > MIN_MASK_PARTIAL:
                out.append(
                    img[
                        top:top+conv_size[1],
                        left:left+conv_size[0]
                    ]
                )
                
    return out

In [8]:
dotenv = load_dotenv()

In [10]:
with sqlite3.connect( dotenv['DB_PATH'] ) as con:
    masks = pd.read_sql(
        '''
            SELECT * FROM mask
        ''',
        con
    )

In [11]:
sample = masks[pd.isna(masks.K_max_freq_difference)].sample(n=100)

In [12]:
def process_row( row ):
    # Daten laden
    non_moire_img_path = get_related_filepath(
        row.job,
        'ps2400dpi150lpi',
        f'{ row.pdf_filename }.4c_{ dotenv["LOFI_DPI"] }.jpg'
    )
    non_moire_img = open_img( non_moire_img_path )
    non_moire_img = non_moire_img.resize((
        round(non_moire_img.size[0] * (dotenv["LOFI_DPI"] / dotenv["TRAIN_DATA_DPI"])),
        round(non_moire_img.size[1] * (dotenv["LOFI_DPI"] / dotenv["TRAIN_DATA_DPI"]))
    ))
    
    moire_img_path = get_generic_image_filepath(
        row.pdf_filename,
        row.job,
        row.method,
        row.idx,
        variant=f'4c_{ dotenv["LOFI_DPI"] }'
    )
    moire_img = open_img( moire_img_path )
    moire_img = moire_img.resize(non_moire_img.size)
    
    masks_path = get_related_filepath(
        row.job,
        f'halftone{ dotenv["LOFI_DPI"] }dpi',
        f'{ row.pdf_filename }.masks.pkl'
    )
    mask_data = load_masks( masks_path )
    
    relevant_mask_data = [
        m for m in mask_data
        if m['bbox'] == [int(val) for val in row.bbox.split(';')]
    ][0]
    
    whole_mask = get_whole_mask(relevant_mask_data)
    whole_mask = cv2.resize(whole_mask, non_moire_img.size, interpolation=cv2.INTER_AREA)

    # cut into tiles
    moire_tiles = cut_img_to_tiles( np.array(moire_img.convert('L')), relevant_mask_data['bbox'], mask=whole_mask )
    non_moire_tiles = cut_img_to_tiles( np.array(non_moire_img.convert('L')), relevant_mask_data['bbox'], mask=whole_mask )
    max_diff = []

    for i in range(len(moire_tiles)):
        out = gaussian_filter(
            np.log((np.abs(get_fft(non_moire_tiles[i])) ** 2) / (np.abs(get_fft(moire_tiles[i])) ** 2)),
            sigma=3
        )
        max_diff.append( out.max() )

    return max(max_diff)

In [13]:
with sqlite3.connect( dotenv['DB_PATH'] ) as con:
    for i in tqdm(range(sample.shape[0])):
        row = sample.iloc[i]
        try:
            max_diff = process_row( row )
        
            masks.loc[
                row.name,
                'K_max_freq_difference'
            ] = max_diff

            c = con.cursor()
            c.execute(
                f'''
                    UPDATE mask
                    SET K_max_freq_difference={ max_diff }
                    WHERE mask_id="{ row.mask_id }"
                '''
            )
            c.close()
            con.commit()
        except:
            pass

  0%|          | 0/100 [00:00<?, ?it/s]

/var/folders/ns/b_3z05zx1bz71dzh3qcvtpyh0000gq/T/ipykernel_48629/1258449078.py:46: RuntimeWarning: invalid value encountered in divide
  np.log((np.abs(get_fft(non_moire_tiles[i])) ** 2) / (np.abs(get_fft(moire_tiles[i])) ** 2)),
/var/folders/ns/b_3z05zx1bz71dzh3qcvtpyh0000gq/T/ipykernel_48629/1258449078.py:46: RuntimeWarning: divide by zero encountered in log
  np.log((np.abs(get_fft(non_moire_tiles[i])) ** 2) / (np.abs(get_fft(moire_tiles[i])) ** 2)),
/var/folders/ns/b_3z05zx1bz71dzh3qcvtpyh0000gq/T/ipykernel_48629/1258449078.py:46: RuntimeWarning: divide by zero encountered in divide
  np.log((np.abs(get_fft(non_moire_tiles[i])) ** 2) / (np.abs(get_fft(moire_tiles[i])) ** 2)),


In [ ]:
masks.loc[
    pd.isna(masks.K_mean_max_freq_difference) == False,
    ['job','pdf_filename','idx','mask_id','ssim','K_mean_max_freq_difference']
]